# 5. COBRA vs. MASS 

When determining the discrepencies between COBRA and MASS, we have to understand the nuances and certain discrepencies between cobrapy and masspy. This documentation compares and contrasts the differences between the two models and discerns how mass action differs from flux balance analysis. We will start at the very begininning, from the getting started documentation and clarify the differences between cobrapy and masspy as we go through each documentation workbook. 

<font color='red'>

* Nice work, I like where you are going with this.

* Instead of using cobra_model and mass_model, use the notations cobra_model and mass_model, cobra_reaction and mass_reaction, etc. (Use prefix cobra_ and mass_ when necessary)
    
* Leave a section for Genes, but skip making any section content for the time being

</font>

## 5.1 Inspection 

### 5.1.1 Importing Model Systems

At the beginning or each workflow, we import the speicifc methods that we wish to utilize for data analysis. For __cobrapy__, usual methodology to import the model and it's corresponding objects within the work environment can be accomplish through the following, 

In [1]:
from __future__ import print_function
from cobra import Model, Reaction, Metabolite
import cobra.test

However, __masspy__ has a slightly different syntax and requires more than just the importing of MASS itself; rather, it needs all of the objects which we utilize, from building models to creations of simulations to visualizing solutions,

In [2]:
from mass import (
    MassModel, MassMetabolite, MassReaction, Simulation, Solution,
    plot_simulation, plot_phase_portrait, plot_tiled_phase_portrait)
from mass.test import create_test_model

Other packages which are vital for information and visualization of data are also imported seperately, but is not seen to be the case for cobrapy, 

In [3]:
import numpy as np
import pandas as pd
import sympy as sym
import matplotlib as mpl
import matplotlib.pyplot as plt

Most of this comes down to the amount of information masspy can derive versus the amount of information cobrapy utilizes. From the basis of cobrapy, COBRA uses constraint based analysis to determine fluxes within and out of a particular system; for such model construction, an immense ammount of information is not required to derive constraints on a system based on it's fluxes. However, masspy requires much more adept analysis as an influx of information is created to determine the mass action stoichiometric solutions of any given model. Whereas COBRA focuses more on the mathematical aspect, MASS constructs models and determines the analytical portion of the model.

### 5.1.2 General breakdown of class, attributes and objects

For this type of analysis, we will create two test models; one is cobra_model and the other is mass_model for cobrapy and masspy models respectively.

In [4]:
cobra_model = cobra.test.create_test_model("textbook")
mass_model = create_test_model("textbook")

In [5]:
cobra_model

Name,e_coli_core
Memory address,0x012c222e48
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


In [6]:
mass_model

Name,Core_RBC_Model
Memory address,0x012cf1a128
Stoichiometric Matrix,48x53
Matrix Rank,44
Matrix Type,"dense, float64"
Number of Metabolites,48
Number of Reactions,53
Number of Initial Conditions,48
Number of Forward Rate Constants,53
Number of Equilibrium Constants,53
Number of Irreversible Reactions,11


More attributes are present within the masspy model, further qualifying the analysis of mass and cobra at the end of 5.1.1. MASS provides more flexibility for determining how the system behaves and allows for control over users to modulate different behaviors and perturbations whereas COBRA allows for the dimensional breakdown of the specific system through constraint based analysis and determination/ optimization of values based on the flux of reactions & metabolites into and out of the system. 

Upon further analysis, when we pull up a specific reaction, we can observe, 

In [7]:
mass_model.reactions[29]

Reaction identifier,DPGM
Name,Diphosphoglyceromutase
Memory address,0x012cd6fb00
Subsystem,Hemoglobin
Stoichiometry,"_13dpg_c <=> _23dpg_c + h_c 3-Phospho-D-glyceroyl phosphate <=> 2,3-Disphospho-D-glycerate + H+"
GPR,
Kinetic Reversibility,True


In [8]:
cobra_model.reactions[29]

Reaction identifier,EX_glu__L_e
Name,L-Glutamate exchange
Memory address,0x012d0c32e8
Stoichiometry,glu__L_e --> L-Glutamate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Within __cobra__, we can distinctly see that an emphasis on lower and upper bounds is placed so as to constrain the reaction. This allows for mathematical calculations to be formulated as the fluxes in and out of a system can be predicated on a certain boundary limit. Thereby, cobrapy works by determining the direction of a reaction (whether it's pushing forward or going back) based on the theoretical limits placed and values of the corresponding fluxes within those limits 

__One major distinction (due to the upper and lower bound constraints) is that there is no creation of initial conditions nor parameters which can be set.__  The constraints themselves act as conditions that must be met and parameters for the system to follow within cobrapy. In stark contrast, we always set up initial conditions for our metabolites and parameters for our reactions to determine solutions for any given model

### 5.1.3 Genes

In [9]:
#Skip 4 now 

## 5.2 Building a Model

### 5.2.1 Model Creation

A lot of the cobrapy method and masspy method for constrcution of a model initially overlaps. Both intiially are similar in that a model creation is required, however, cobrapy construction creates the reaction first but not the metabolites. Masspy construction begins with the creation of metabolites first and then the reactions, 

In [10]:
#Create CobraModel
cobra_model = Model('example_model')

#Generate CobraReactions
cobra_reaction = Reaction('3OAS140')
cobra_reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
cobra_reaction.subsystem = 'Cell Envelope Biosynthesis'
cobra_reaction.lower_bound = 0.  # Default
cobra_reaction.upper_bound = 1000.  #Default

In [11]:
# Create MassModel
mass_model = MassModel('Toy_Model')
mass_model.description = 'Example Model used to Describe Simple Irreversible 2 Step Reaction'
# Generate the MassMetabolites 
x1 = MassMetabolite('x1')
x2 = MassMetabolite('x2')
x3 = MassMetabolite('x3')
# Generate the MassReactions 
v1 = MassReaction("v1")
v2 = MassReaction("v2", reversible=False)

It isn't necessary to create metabolites and reactions in that order when it comes to masspy, but since we are building the model from the bottom up, we begin with rudimentary objects and then follow a chronological process of more complex objects till we create the necessary metabolites and corresponding reactions.

Cobrapy is inherently different due to constrain based analysis; it follows a top-down system and thereby is more focused on the construction of the reactions first in order to determine the specific intricacies of the flux into and out of the system. Furthermore, setting the constrains and bounds of the reactions further helps to corroborate this top down mechanism; naturally, cobrapy add metabolites after the addition of reactions,

In [12]:
#Generate CobraMetabolites
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    '3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

The detail for each metabolite in cobrapy is much more meticulous than the documentation used in masspy; from specifying the name and formula to determining which compartment the metabolites belong to, cobrapy hones in to specify intricacies of it's metabolites. 

### 5.2.2 Adding Metabolites to Reactions

Both cobrapy and masspy add metabolites to reactions and then reactions to the model; there is no discrepency between the two procedures nor methodologies,

In [13]:
#COBRA Method
cobra_reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

cobra_reaction.reaction  # String output of reaction

'ddcaACP_c + h_c + malACP_c --> 3omrsACP_c + ACP_c + co2_c'

*Adding Genes to the Cobra Model*

In [14]:
#skip
cobra_reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
cobra_reaction.genes

frozenset({<Gene STM1197 at 0x12cf97e80>, <Gene STM2378 at 0x12cf974e0>})

In [15]:
pass

In [16]:
#Add MassMetabolites to Rxns
v1.add_metabolites({x1 : -1, x2 : 1})
v2.add_metabolites({x2 : -1, x3 : 1})

Thereafter, reactions are added to the model in a similar manner for both cobrapy and masspy

In [17]:
cobra_model.add_reactions([cobra_reaction])

In [18]:
mass_model.add_reactions([v1,v2])

In [19]:
cobra_model

Name,example_model
Memory address,0x012cf97978
Number of metabolites,6
Number of reactions,1
Objective expression,0
Compartments,c


In [20]:
mass_model

Name,Toy_Model
Memory address,0x012ced46d8
Stoichiometric Matrix,3x2
Matrix Rank,2
Matrix Type,"dense, float64"
Number of Metabolites,3
Number of Reactions,2
Number of Initial Conditions,0
Number of Forward Rate Constants,0
Number of Equilibrium Constants,1
Number of Irreversible Reactions,1


### 5.2.3 Objectives in Cobrapy

The format of creating models is quite similar for mass and cobra; genes are the only section which are a little bit more intuitive; however, there is also the addition of an objective which cobra utilizes which is essentially the maximization of flux for the cobra model, 

In [21]:
cobra_model.objective = '3OAS140'

Which can be examined by, 

In [25]:
print(cobra_model.objective.expression)
print(cobra_model.objective.direction)

-1.0*3OAS140_reverse_65ddc + 1.0*3OAS140
max


## 5.3 Simulations & Visualiation

Simulations between cobrapy and masspy are extremely different; one is a mathematical modeling system comprised of determining fluxes based on constraints while the other focuses on the mass action stoichiometric system as a dynamic entity as it's own. From here, we see that the discrepency of how cobrapy simulates functions is based on flux optimization and constraint based analysis. Masspy can be contrasted upon, but cannot be compared to in detail due to the wide range of discrepency between the two methods of modeling. 

Furthermore, it is interesting to note that *cobrapy has no method to visualize data that is printed to us*. Most of the data is the raw presentation of data in the form of tables. This difference points to the more interesting nuances between the two modeling systems. 

*Note: We will not analyze cobrapy against masspy due to the great variation between the two models; rather, we advise individuals to read upon the simulation and analysis COBRA utilizes from their documentation page: https://cobrapy.readthedocs.io/en/latest/simulating.html * Our methods 